In [99]:
import csv
import pandas as pd
from sklearn import tree
import pydotplus
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import matplotlib.image as pltimg
from sklearn.preprocessing import OneHotEncoder

In [109]:
def mapping(df):
    df_values = df.values
    dict_to_map = {}
    count = 0
    for i in df_values:
        if (i not in dict_to_map):
            dict_to_map[i] = count
            count = count+1
    return dict_to_map

In [136]:
map_grau_de_risco = {
    'Alto Risco': 1,
    'Baixo Risco': 0
}

In [138]:
map_segmento = {
    'PRODUTOS': 1,
    'SERVIÇOS': 0
}

In [158]:
map_licenciados = {
    'Não Licenciados': 0,
    'Em processo': 1,
    'Licenciados' : 2
}

In [157]:
y['label_licenciados'].unique()

array(['Em processo', 'Não Licenciados', 'Licenciados'], dtype=object)

In [137]:
db['SEGMENTO DA ATIVIDADE ECONÔMICA'].unique()

array(['PRODUTOS', 'SERVIÇOS'], dtype=object)

In [150]:
db = pd.read_csv('./db_final.csv')

In [151]:
features = ['Grau de Risco', 'SEGMENTO DA ATIVIDADE ECONÔMICA']
target = ['label_licenciados']

In [152]:
output_regiao = pd.get_dummies(db['Classificação Manual'], prefix='regiao')
output_subgrupo = pd.get_dummies(db['SUBGRUPO'], prefix='subgrupo')

In [153]:
db = pd.concat([db, output_regiao], axis = 1)
db = pd.concat([db, output_subgrupo], axis = 1)

In [154]:
for i in output_regiao.columns:
    features.append(i)
for i in output_subgrupo.columns:
    features.append(i)

In [155]:
features

['Grau de Risco',
 'SEGMENTO DA ATIVIDADE ECONÔMICA',
 'regiao_Centro-Oeste',
 'regiao_Centro-Sul',
 'regiao_Leste',
 'regiao_Norte',
 'regiao_Oeste',
 'regiao_Sul',
 'subgrupo_ALIMENTOS',
 'subgrupo_ASSISTÊNCIA À SAÚDE',
 'subgrupo_COSMÉTICOS',
 'subgrupo_DIVERSOS',
 'subgrupo_INTERESSE Á SAÚDE',
 'subgrupo_MEDICAMENTOS',
 'subgrupo_PPS',
 'subgrupo_SANEANTES']

In [156]:
X = db[features]
y = db[target]

In [130]:
X

,Grau de Risco,SEGMENTO DA ATIVIDADE ECONÔMICA,regiao_Centro-Oeste,regiao_Centro-Oeste,regiao_Centro-Sul,regiao_Centro-Sul,regiao_Leste,regiao_Leste,regiao_Norte,regiao_Norte,...,regiao_Sul,regiao_Sul,subgrupo_ALIMENTOS,subgrupo_ASSISTÊNCIA À SAÚDE,subgrupo_COSMÉTICOS,subgrupo_DIVERSOS,subgrupo_INTERESSE Á SAÚDE,subgrupo_MEDICAMENTOS,subgrupo_PPS,subgrupo_SANEANTES
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,0,0,0,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9530,0,1,0,0,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9531,0,0,0,0,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9532,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9533,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [159]:
X['Grau de Risco'] = X['Grau de Risco'].map(map_grau_de_risco)
X['SEGMENTO DA ATIVIDADE ECONÔMICA'] = X['SEGMENTO DA ATIVIDADE ECONÔMICA'].map(map_segmento)
y['label_licenciados'] = y['label_licenciados'].map(map_licenciados)

<ipython-input-159-bbd7ddce3776>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Grau de Risco'] = X['Grau de Risco'].map(map_grau_de_risco)
<ipython-input-159-bbd7ddce3776>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['SEGMENTO DA ATIVIDADE ECONÔMICA'] = X['SEGMENTO DA ATIVIDADE ECONÔMICA'].map(map_segmento)
<ipython-input-159-bbd7ddce3776>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [167]:
dtree = DecisionTreeClassifier()
dtree = dtree.fit(X, y)
data = tree.export_graphviz(dtree, out_file=None, feature_names=features)
graph = pydotplus.graph_from_dot_data(data)
graph.write_png('decision_tree.png')

True

# Regressão Logística

In [163]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

In [164]:
model = LogisticRegression(multi_class='multinomial', solver = 'lbfgs')

fitting = model.fit(X, y)

/home/igoromote/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/igoromote/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [165]:
fitting.score(X,y)

0.5705296276874672

In [166]:
fitting.coef_

array([[-2.87306223e-01, -6.62730031e-02,  2.81545342e-02,
        -1.16994391e-01,  8.56825131e-02,  6.21641492e-02,
        -1.92859395e-02, -2.83902223e-02,  3.01991652e-02,
         1.46001781e-01, -2.98479736e-01,  2.15805546e-01,
        -6.83981338e-02, -1.90342130e-01, -4.60984952e-02,
         2.22642648e-01],
       [ 1.49002367e+00, -2.64615441e-02, -7.73236050e-02,
         6.85162889e-02,  4.63211693e-04, -9.40232469e-02,
         1.05872354e-02,  6.56131395e-02, -1.68598361e-01,
        -1.21859758e-01,  3.63565427e-01, -1.37371027e-01,
         1.22154326e-01, -4.39934848e-02,  2.64885574e-02,
        -6.65526555e-02],
       [-1.20271745e+00,  9.27345471e-02,  4.91690709e-02,
         4.84781020e-02, -8.61457248e-02,  3.18590977e-02,
         8.69870410e-03, -3.72229172e-02,  1.38399196e-01,
        -2.41420227e-02, -6.50856903e-02, -7.84345188e-02,
        -5.37561919e-02,  2.34335615e-01,  1.96099378e-02,
        -1.56089992e-01]])